> First time use: follow instructions in the README.md file in this directory.



# Receitas

Várias receitas para usar Timelink com notebooks

[Sumário]

---

# Receipts

Receipts for using Timelink with notebooks

[summary]


---
## Setup

### Basic setup

Create a `TimelinkNotebook` instance.

This will provide an interface to the 
various `Timelink` functions.



In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()


### Changing the default values

When called with no parameters `TimelinkNotebook` 
assumes a certain directory layout containing the notebook
and from that layout autoconfigures various parameters.

The default layout:

* **project-directory**
    * **database** (database related filed)
    * **notebooks**  _(directory with the current notebook)_ 
    * **kleio**  (source files in kleio format)
    * **identifications** (files with identification information)

Based on this layout `TimelinkNotebook` assumes a few default values:

* **database name**
    * the name of the *project directory* is used as the **database name**, sanitized to produce a valid database name
      (hifens and spaces are replaced with _ ).
* **database type**
  * the type of the database is assumed to be **sqlite**  and a **sqlite** database is created in 
      *database/sqlite*
    * the `db_type` parameter can be used to set the database to `postgres`. 
* **Kleio server home**, working directory for the _Kleio server_ is assumed to be the **project directory**
  * The _Kleio server_ is an application
    that processes transcriptions of historical _Kleio_ notation and generates data in a format
    that can be imported to the database.

These default values can be changed with parameters when creating the `TimelinkNotebook`  instance:

```python
tlnb = TimelinkNotebook(
    project_name="my-project",
    project_home="~/projects/m-project",
    db_type="postgres",
    db_name="my_project_db",
    kleio_image="timelinkserver/kleio-server",
    kleio_version="12.0.0",
    postgres_image="postgres",
    postgres_version="latest",
    sqlite_dir="~/databases",
    sql_echo=True
)
```

### Examining the current configuration

In [2]:

tlnb.print_info()


Project name: tutorial
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: tutorial
Kleio image: timelinkserver/kleio-server
Kleio version: latest
Kleio server token: yNEupdoWvbKkAfALXitPVuC0D9PXUkPa
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
TimelinkNotebook(project_name=tutorial, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=tutorial, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


### Listing available databases

In [3]:
pd = tlnb.get_postgres_databases()
print(f"Postgres databases: {pd}")
sd = tlnb.get_sqlite_databases()
print(f"Sqlite databases: {sd}")


Postgres databases: ['timelink', 'tests']
Sqlite databases: ['../database/sqlite/tutorial.sqlite']


### Database status (table row count)

In [4]:
tlnb.table_row_count_df()

,table,count
0,acts,3
1,attributes,14680
2,class_attributes,63
3,classes,13
4,entities,17182
5,geoentities,215
6,goods,0
7,kleiofiles,3
8,objects,0
9,persons,873


## Dealing with kleio files

### List Kleio files available

In [5]:
kleio_files = tlnb.get_kleio_files()
kleio_files.head()

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,kleio/auc-alunos.cli,auc-alunos.cli,2024-02-04 08:38:20.975128+00:00,V,2024-02-04 08:38:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-04 08:38:54.291236,/rest/reports/kleio/auc-alunos.rpt,/rest/exports/kleio/auc-alunos.xml
1,kleio/dehergne-a.cli,dehergne-a.cli,2024-02-04 14:08:29.476407+00:00,V,2024-02-04 14:08:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-04 14:08:36.475775,/rest/reports/kleio/dehergne-a.rpt,/rest/exports/kleio/dehergne-a.xml
2,kleio/dehergne-locations-1644.cli,dehergne-locations-1644.cli,2024-02-04 08:40:21.505312+00:00,V,2024-02-04 08:40:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-04 08:40:24.149131,/rest/reports/kleio/dehergne-locations-1644.rpt,/rest/exports/kleio/dehergne-locations-1644.xml


Show only translation and import status

In [6]:
kleio_files[["name","status","import_status"]]

,name,status,import_status
0,auc-alunos.cli,V,I
1,dehergne-a.cli,V,I
2,dehergne-locations-1644.cli,V,I


Show translation and import errors

In [7]:
kleio_files[["name","errors","warnings","import_errors","import_warnings"]]

,name,errors,warnings,import_errors,import_warnings
0,auc-alunos.cli,0,0,0,0
1,dehergne-a.cli,0,0,0,0
2,dehergne-locations-1644.cli,0,0,0,0


### Translation and Import reports

Translation reports.

Pass a dataframe with Kleio files and a row number to get the translation report

In [8]:
rpt=tlnb.get_translation_report(kleio_files,rows=1)
print(rpt)

KleioTranslator - server version 12.2 - build 560 2024-01-08 08:18:15
4-2-2024 14-8

Processing data file dehergne-a.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 178
Obs: 
** Processing source fonte$dehergne-a
7: lista$dehergne-notices-a
*** End of File

Line 541 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-belchior-miguel-carneiro-leitao) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 814 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-jean-regis-lieou) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 876 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-alessandro-cicero) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.

Structure file: /usr/local/timelink/clio/src/gacto2.str
Structure processing report: /usr/local/timelink/clio/src/gacto2.srpt
Structure in JSON: /usr/loc

Or use the file name

In [9]:
file = kleio_files.iloc[0].rpt_url
print(file)
rpt=tlnb.get_translation_report(file)
print(rpt)

/rest/reports/kleio/auc-alunos.rpt
KleioTranslator - server version 12.2 - build 560 2024-01-08 08:18:15
4-2-2024 8-37

Processing data file auc-alunos.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 6
Obs: 
** Processing source fonte$auc-alunos
30: lista$auc-alumni-A-140337-140771
*** End of File


Structure file: /usr/local/timelink/clio/src/gacto2.str
Structure processing report: /usr/local/timelink/clio/src/gacto2.srpt
Structure in JSON: /usr/local/timelink/clio/src/gacto2.str.json

Kleio file: /kleio-home/kleio/auc-alunos.cli
Original file: /kleio-home/kleio/auc-alunos.org
Previous version: /kleio-home/kleio/auc-alunos.old
Temp file with ids: /kleio-home/kleio/auc-alunos.ids
0  errors. 
0  warnings.
Translation finished.



Import report

In [10]:
rpt = tlnb.get_import_rpt(kleio_files,rows=1)
print(rpt)

No errors
No warnings



# Atualizar base de dados

Atualiza traduções de fontes e importa quando tradução não tem erros.

---

# Update database

Updates source translations and imports into database sources with no errors.

In [11]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

Show imported files status

In [12]:
imported_files_df = tlnb.get_import_status()
imported_files_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   path                3 non-null      object             
 1   name                3 non-null      object             
 2   size                3 non-null      int64              
 3   directory           3 non-null      object             
 4   modified            3 non-null      datetime64[ns, UTC]
 5   modified_iso        3 non-null      datetime64[ns, UTC]
 6   modified_string     3 non-null      object             
 7   qtime               3 non-null      datetime64[ns, UTC]
 8   qtime_string        3 non-null      object             
 9   source_url          3 non-null      object             
 10  status              3 non-null      object             
 11  translated          3 non-null      datetime64[ns, UTC]
 12  translated_string   3 non-null      obje

Check the import status of the translated files
```python
    I = "I" # imported
    E = "E" # imported with error
    W = "W" # imported with warnings no errors
    N = "N" # not imported
    U = "U" # translation updated need to reimport

``` 

In [13]:
imported_files_df[["import_status","import_errors","import_warnings","name","imported","path"]].sort_values("name")

,import_status,import_errors,import_warnings,name,imported,path
0,I,0,0,auc-alunos.cli,2024-02-04 08:38:54.291236,kleio/auc-alunos.cli
1,I,0,0,dehergne-a.cli,2024-02-04 14:08:36.475775,kleio/dehergne-a.cli
2,I,0,0,dehergne-locations-1644.cli,2024-02-04 08:40:24.149131,kleio/dehergne-locations-1644.cli


## Todo

This as data frame in a single function
* TimelinkNotebook.translate([files_df,rows=List[int],status="T"])
* TimelinkNotebook.import(files_df, rows=List[int])



# Obter dados
---

# Getting data

### Procurar pessoa

---
### Search for people

#### Search persons by name

In [14]:
from timelink.api.models import Person
from sqlalchemy import select

show_only=10

with tlnb.db.session() as session:
    stmt = select(Person).where(Person.name.like('% Aboim'))
    print(stmt)
    persons = session.execute(stmt).scalars().all()
    print()
    for person in persons[:show_only]:
        print(person.id, person.name,person.sex)

SELECT persons.id, entities.id AS id_1, entities.class, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE persons.name LIKE :name_1

268593 José Inácio Correia Manuel de Aboim m
140347 Adrião de Aboim m
268598 Manuel de Azevedo Guerreiro de Aboim m
140348 Alberto António de Brito Guerreiro e Aboim m
140348-pai António de Brito Guerreiro Aboim m
140349 António de Aboim m
140350 António Camacho Guerreiro de Brito e Aboim m
140351 António Pestana Guerreiro Camacho Aboim m
140352 David Vaz da Fonseca Aboim m
140353 Diogo Guerreiro Camacho de Aboim m


### Search other Entities

#### get the Entity classes in the database

In [15]:
from sqlalchemy import select, func
from timelink.api.models import Entity

models = tlnb.db.get_models_ids()
models

['attribute',
 'relation',
 'act',
 'source',
 'person',
 'good',
 'object',
 'class',
 'geoentity',
 'rgeoentity',
 'robject',
 'rperson',
 'rentity',
 'entity']

#### Get columns of an entity type

In [16]:
table = tlnb.db.get_table("entity")
print(table.name)
list(table.columns)

entities


[Column('id', String(), table=<entities>, primary_key=True, nullable=False),
 Column('class', String(), table=<entities>, nullable=False),
 Column('inside', String(), ForeignKey('entities.id'), table=<entities>),
 Column('the_order', Integer(), table=<entities>),
 Column('the_level', Integer(), table=<entities>),
 Column('the_line', Integer(), table=<entities>),
 Column('groupname', String(), table=<entities>),
 Column('updated', DateTime(), table=<entities>, default=CallableColumnDefault(<function datetime.utcnow at 0x144b67010>)),
 Column('indexed', DateTime(), table=<entities>)]

#### Search any entity type

## IMPROVE

In [17]:
from timelink.api.models import Entity
from sqlalchemy import select, func

Geoentity = tlnb.db.get_model("geoentity")
stmt = select(Geoentity).where(Geoentity.name.like('H%'))
print(stmt)
with tlnb.db.session() as session:

    result = session.execute(stmt).scalars().all()
    for row in result:
        print()
        print(row.the_type,row.name,row.id,row.obs)
        print(row.to_kleio())

SELECT geoentities.id, entities.id AS id_1, entities.class, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, geoentities.name, geoentities.obs, geoentities.the_type 
FROM entities JOIN geoentities ON entities.id = geoentities.id 
WHERE geoentities.name LIKE :name_1

geo2 Hangchou deh-r1644-hangchou None
geo2$deh-r1644-hangchou/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$activa/sim/16110000
  atr$residencia-missao/Jesuíta/16110000
  geo3$deh-r1644-fuyang/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/16420000
  geo3$deh-r1644-jenho/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/16080000

geo2 Huchow deh-r1644-huchow None
geo2$deh-r1644-huchow/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  geo3$deh-r1644-tehtsing/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    

#### Get person by id

Show a single person or entity in Kleio notation

In [18]:
p = tlnb.db.get_person(person.id)
print(p.to_kleio())

n$Diogo Guerreiro Camacho de Aboim/m/id=140353/obs="""
      """

                  Id: 140353
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000019

                  Nome        : Diogo Guerreiro Camacho de Aboim
                  Data inicial: 1721-10-01
                  Data final  : 1727-10-01
                  Filiação: Manuel Guerreiro Camacho

                  Naturalidade: Padrões
                  Faculdade: Cânones

                  Matrícula(s): 01.10.1722
                  01.10.1723
                  01.10.1724
                  01.10.1725
                  01.10.1726
                  01.10.1727
                  Instituta: 01.10.1721
                  Bacharel: 31.05.1726
                  Formatura:
                  Licenciado: 28.06.1727
                  Doutor:

                  Outras informações:
                  Suficiência: 11.06.1726
                  Repetiação: 21.06.1727
                  Exame Privado: 28.06.1727
              "

### Show other type of entities by id in Kleio

In [19]:
from timelink.api.models import Entity

ent = tlnb.db.get_entity("deh-r1644-chekiang")
print(ent.to_kleio())

geo1$deh-r1644-chekiang/type=geoentity
  rel$function-in-act/geo1/deh-chre-1644/16440000
  geo2$deh-r1644-hangchou/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    atr$activa/sim/16110000
    atr$residencia-missao/Jesuíta/16110000
    geo3$deh-r1644-fuyang/type=geoentity
      rel$function-in-act/geo3/deh-chre-1644/16440000
      atr$activa/sim/16420000
    geo3$deh-r1644-jenho/type=geoentity
      rel$function-in-act/geo3/deh-chre-1644/16440000
      atr$activa/sim/16080000
  geo2$deh-r1644-chuchow/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    atr$activa/sim/16130000
  geo2$deh-r1644-huchow/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    geo3$deh-r1644-tehtsing/type=geoentity
      rel$function-in-act/geo3/deh-chre-1644/16440000
      atr$activa/sim/1620
  geo2$deh-r1644-kashing/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    geo3$deh-r1644-kaosham/type=geoentity
      rel$function-in-act/geo3



###  Obter um dataframe a partir de atributos

---


###  Get a Dataframe from attributes


#### Exemplo: Faculdade, data de entrada e data de saída e grau dos naturais de Coimbra

In [20]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='person',
                    the_type='naturalidade',
                    the_value='Coimbra',
                    more_info=['name','sex'],
                    name_like='% Aboim',
                    more_cols=['faculdade','uc.entrada','uc.saida'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 140349 to 140367
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               2 non-null      object
 1   sex                2 non-null      object
 2   naturalidade       2 non-null      object
 3   naturalidade.date  2 non-null      object
 4   naturalidade.obs   0 non-null      object
 5   faculdade          2 non-null      object
 6   faculdade.date     2 non-null      object
 7   faculdade.obs      0 non-null      object
 8   uc.entrada         2 non-null      object
 9   uc.entrada.date    2 non-null      object
 10  uc.entrada.obs     0 non-null      object
 11  uc.saida           2 non-null      object
 12  uc.saida.date      2 non-null      object
 13  uc.saida.obs       0 non-null      object
dtypes: object(14)
memory usage: 348.0+ bytes


In [21]:
df.head(5)

,name,sex,naturalidade,naturalidade.date,naturalidade.obs,faculdade,faculdade.date,faculdade.obs,uc.entrada,uc.entrada.date,uc.entrada.obs,uc.saida,uc.saida.date,uc.saida.obs
id,,,,,,,,,,,,,,
140349,António de Aboim,m,Coimbra,1566-12-20,None,Cânones,1566-12-20,None,1566-12-20,1566-12-20,None,1574-07-24,1574-07-24,None
140367,Manuel de Vargas de Aboim,m,Coimbra,20200211,None,Cânones,20200211,None,0000-00-00,20200211,None,0000-00-00,20200211,None


#### obter attributos de outras entidades

In [22]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='geoentity',
                    more_info=['name'],
                    the_type='activa',
                    the_value='sim',
                    more_cols=['residencia-missao'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, deh-r1644-anhai to deh-r1644-yunnan
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    165 non-null    object
 1   activa                  165 non-null    object
 2   activa.date             165 non-null    object
 3   activa.obs              4 non-null      object
 4   residencia-missao       38 non-null     object
 5   residencia-missao.date  38 non-null     object
 6   residencia-missao.obs   4 non-null      object
dtypes: object(7)
memory usage: 10.3+ KB


In [23]:
df.head(30)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None
deh-r1644-chala,Chala,sim,1610,None,NaN,NaN,NaN
deh-r1644-changchow-fou,Changchow,sim,1643,None,NaN,NaN,NaN
deh-r1644-changshu,Changshu,sim,1623,None,Jesuíta,1635,"R 1635 «Cham Xo», cf AHSI 28 (1951) 311-312"
deh-r1644-chengting,Chengting,sim,1621,None,NaN,NaN,NaN




###  Remover colunas sem valores

---



###  Remove empty columns

In [24]:
df.dropna(how='all', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, deh-r1644-anhai to deh-r1644-yunnan
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    165 non-null    object
 1   activa                  165 non-null    object
 2   activa.date             165 non-null    object
 3   activa.obs              4 non-null      object
 4   residencia-missao       38 non-null     object
 5   residencia-missao.date  38 non-null     object
 6   residencia-missao.obs   4 non-null      object
dtypes: object(7)
memory usage: 10.3+ KB


In [25]:
df.head(5)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None


In [26]:
ids = df.index.unique()
for id in ids[:5]:
    ent = tlnb.db.get_entity(id)
    print(ent.to_kleio())
    print()

geo3$deh-r1644-anhai/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1634

geo3$deh-r1644-bankao/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1636

geo2$deh-r1644-cantao/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$activa/sim/1555
  atr$residencia-missao/Jesuíta/1580
  atr$residencia-missao/Franciscanos/0000
  atr$residencia-missao/Dominicanos/0000
  geo3$deh-r1644-tungkun/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
  geo3$deh-r1644-quon-yao/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/1621
  geo3$deh-r1644-lampacao/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/1535

geo3$deh-r1644-chala/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1610

geo2$deh-r1644-changchow-fou/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$activa/sim/1643



## Contagens

---

## Counting



###  Contagem de atributos a partir de uma tabela em memória

---

###  Count attributes from an existing dataframe



In [27]:
# create a column with the index values which are the id numbers
df['id'] = df.index.values

col = 'faculdade' # subotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc.entrada':'min',
                                                  'uc.saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

KeyError: 'faculdade'


### Contagens na base de dados

Quando o atributo tem muitos valores e não é necessário
ter todas as pessoas em memória: contagem feita na base de dados

---

### Counting directly in the database
When there are many values and it is not
necessary to have all the people in memory:
count directly in the database.




In [ ]:
from timelink.pandas import attribute_values

df_totals = attribute_values('grau',db=tlnb.db)


In [ ]:
df_totals.head(10)


,count,date_min,date_max
value,,,
Bacharel,170,1554-07-19,1912-08-03
Formatura,153,1574-07-24,1905-06-19
Licenciado,33,1574-06-03,1886-02-27
Bacharel em Artes,25,1574-03-14,1766-07-19
Doutor,11,1560-12-22,1887-11-27
Licenciado em Artes,5,1574-05-15,1738-06-17
Mestre,2,1710-10-05,1768-10-23


#### Filtrar por datas

Para evitar remissivas com data zero

---

#### Filter by dates

Avoid cross-references with zero date

##ERROR

In [ ]:
df_totals = attribute_values('grau',dates_between=('1535','1919'),db=tlnb.db)

In [ ]:
df_totals.head(10)

,count,date_min,date_max
value,,,
Bacharel,170,1554-07-19,1912-08-03
Formatura,153,1574-07-24,1905-06-19
Licenciado,33,1574-06-03,1886-02-27
Bacharel em Artes,25,1574-03-14,1766-07-19
Doutor,11,1560-12-22,1887-11-27
Licenciado em Artes,5,1574-05-15,1738-06-17
Mestre,2,1710-10-05,1768-10-23


## Visualizar registos

---

## View records





### Visualizar uma pessoa

---

### View a person


#### Atributos de uma pessoa numa tabela, uma linha por attributo

---

#### Person attributes in a dataframe, one line per attribute

In [ ]:
import pandas as pd
from timelink.pandas import group_attributes as person_attributes

pd.set_option('display.max_rows',1000)

id = '140349'
pdf = person_attributes([id],db=tlnb.db)  # note id in a list
pdf[['date','type','value','attr_obs']].sort_values(['date','type'])

,date,type,value,attr_obs
id,,,,
140349,1566-12-20,faculdade,Cânones,None
140349,1566-12-20,faculdade.ano,Cânones.1566,None
140349,1566-12-20,naturalidade,Coimbra,None
140349,1566-12-20,naturalidade.ano,Coimbra.1566,None
140349,1566-12-20,nome,António de Aboim,None
140349,1566-12-20,nome-geografico,Coimbra,None
140349,1566-12-20,nome-geografico.ano,Coimbra.1566,None
140349,1566-12-20,nome.apelido,Aboim,None
140349,1566-12-20,nome.primeiro,António,None


#### Atributos de uma pessoa numa tabela, attributos em colunas

---

#### Person attributes in a dataframe, attributes in columns

In [30]:
# Get list of people with with a certain value in a specific attribute

df = entities_with_attribute(
                    entity_type='person',
                    the_type='uc.entrada',  # we need a base attribute
                    more_info=['name','sex'],
                    more_cols=['instituta','matricula.faculdade','matricula.ano'],
                    filter_by=[140349],
                    db=tlnb.db,
                    sql_echo=False)
view_cols = ['name','matricula.ano.date','matricula.ano','matricula.ano.obs']
df[view_cols].sort_values('matricula.ano.date')

,name,matricula.ano.date,matricula.ano,matricula.ano.obs
id,,,,
140349,António de Aboim,1571-07-20,Cânones.1571,20.07.1571
140349,António de Aboim,1573-10-07,Cânones.1573,07.10.1573
140349,António de Aboim,1574-07-24,Cânones.1574,24.07.1574


#### Examinar potenciais duplicados

---

#### Examine potentital duplicates

In [15]:
from timelink.pandas import display_group_attributes
pd.set_option('display.max_rows',250)

no_show=['código-de-referência','data-do-registo','url','faculdade.ano','naturalidade.ano',
         'matricula-faculdade.ano','nome-apelido','nome-primeiro','nome-geografico.ano',
         'grau.ano','matricula-outra.ano','nome-geografico','instituta.ano']

dup_ids = ['234295','234710',]  # Alexandre Metelo de

display_group_attributes(dup_ids,
                             header_cols=['uc-entrada','naturalidade','faculdade','nome-pai'],
                             exclude_attributes=no_show,
                             sort_attributes=['date','type','value'],
                             cmap_name='Pastel1')

,id,uc-entrada,naturalidade,faculdade,nome-pai
0,234295,1704-11-07,Marialva,Cânones,nan
1,234710,1705-10-24,Marialva,Matemática,Manuel Cardoso Metelo


,date,id,type,value,attr_obs
0,1704-11-07,234295,faculdade,Cânones,Cânones
1,1704-11-07,234295,instituta,1704-11-07,07.11.1704 1704-11-07
2,1704-11-07,234295,naturalidade,Marialva,None
3,1704-11-07,234295,nome,Alexandre Metelo de Sousa,None
4,1704-11-07,234295,uc-entrada,1704-11-07,None
5,1704-11-07,234295,uc-entrada.ano,1704,None
6,1705-10-24,234710,faculdade,Matemática,Matemática
7,1705-10-24,234295,matricula-faculdade,Cânones,24.10.1705
8,1705-10-24,234710,matricula-faculdade,Matemática,24.10.1705
9,1705-10-24,234710,naturalidade,Marialva,None


#### Notação Kleio

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation

See [Kleio notation](README_kleio.md)

#### Notação Kleio directamente da base de dados

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation directly from database

See [Kleio notation](README_kleio.md)

In [33]:
from timelink.mhk.models.person import Person

with tlnb.db.session() as session:

    p: Person = session.query(Person).order_by(Person.id).first()
    k = p.to_kleio()
    print(p.to_kleio())


n$António Pinto Abadeço/m/id=140337/obs="""
      """

                  Id: 140337
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000001

                  Nome        : António Pinto Abadeço
                  Data inicial: 1705-10-01
                  Data final  : 1710-10-01
                  Filiação: António Pinto

                  Naturalidade: Abrantes
                  Faculdade: Cânones

                  Matrícula(s): 01.10.1705
                  01.10.1706
                  01.01.1707
                  01.10.1708
                  01.10.1709
                  01.10.1710
                  Instituta:
                  Bacharel: 28.06.1709
                  Formatura:
                  Licenciado:
                  Doutor:

                  Outras informações:
              """
  """
  rel$function-in-act/n/António Pinto Abadeço/auc-alumni-A-140337-140771/20200211
  atr$código-de-referência/""PT/AUC/ELU/UC-AUC/B/001-001/A/000001""/2021-05-17
  atr$data-